# Answering Business Questions using SQL

*Chinook* is a fictional digital music shop, similar to iTunes store. The Chinook database contains tables that include information about the products (i.e. songs or albums) sold, employees, customers, and sales. We will be analyzing these business data in order to provide recommendations to Chinook.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## Data overview

In [2]:
%%sql

SELECT name, 
       type
  FROM sqlite_master
 WHERE type IN ("table", "view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


There are 11 tables in the Chinook database. 

1. **album**: This table contains an information of each album, i.e. name and artist.
2. **artist**: This table stores artist names. Each artist has their own ID.
3. **customer**: This table stores customers' information and the ID of sales agent assigned to each customer.
4. **employee**: This table stores employees' information.
5. **genre**: This table stores a list of music genre.
6. **invoice**: This table provides header data of each invoice. 
7. **invoice_line**: This table provides data of each invoice line, including unit price and quantity.
8. **media_type**: This table stores a list of media type.
9. **playlist**: This table stores a list of playlists created in Chinook store.
10. **playlist_track**: This table provides information of which tracks are added to each playlist.
11. **track**: This table contains information of each track available in Chinook store.

The database schema is provided in the below picture.

![chinook_schema](pics\schema.png)

## 1
## Which albums should Chinook add to the store?

**Scenario**

>The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

>| Artist name | Genre |
|-------------|-------|
| Regal | Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

>The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA.

**Question**

Which artist should Chinook select to add to the store? 

In [3]:
%%sql

WITH invoice_genre_usa AS (SELECT c.country,
                                  il.track_id,
                                  t.name track_name,
                                  g.name genre
                             FROM customer c
                                  INNER JOIN invoice i ON c.customer_id = i.customer_id
                                  INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
                                  INNER JOIN track t ON il.track_id = t.track_id
                                  INNER JOIN genre g ON t.genre_id = g.genre_id 
                                  WHERE country == 'USA')

SELECT genre,
       COUNT(track_name) num_tracks_sold,
       ROUND(CAST(COUNT(track_name) AS FLOAT) / (SELECT COUNT(track_name)
                                                   FROM invoice_genre_usa), 2) pct_tracks_sold
  FROM invoice_genre_usa igu
 GROUP BY genre
 ORDER BY num_tracks_sold DESC
 LIMIT 10;

 * sqlite:///chinook.db
Done.


genre,num_tracks_sold,pct_tracks_sold
Rock,561,0.53
Alternative & Punk,130,0.12
Metal,124,0.12
R&B/Soul,53,0.05
Blues,36,0.03
Alternative,35,0.03
Pop,22,0.02
Latin,22,0.02
Hip Hop/Rap,20,0.02
Jazz,14,0.01


**Conclusion**

Chinook should select albums of *Red Tone*, *Slim Jim Bites*, and *Meteor and the Girls* to add to the stores.

**Analysis**

When we rank the genre of the four new artists by percentage of tracks sold, punk music comes first, followed by blues, pop, and hip-hop. Chinook should pick the top 3 genres that are sold the most.

**Recommendation**

The percentage of the three selected genres combined only account for 17% of all tracks sold in USA. Chinook might consider focusing more on finding new rock artist(s) to add to the stores.

## 2
## Which genre is the most popular in Chinook?

Now that we already take a look at USA market, we will take one step back to see the popularity of each genre for the whole Chinook store.

In [4]:
%%sql

WITH invoice_genre AS (SELECT c.country,
                              il.track_id,
                              t.name track_name,
                              g.name genre
                         FROM customer c
                              INNER JOIN invoice i ON c.customer_id = i.customer_id
                              INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
                              INNER JOIN track t ON il.track_id = t.track_id
                              INNER JOIN genre g ON t.genre_id = g.genre_id)

SELECT genre,
       COUNT(track_name) num_tracks_sold,
       ROUND(CAST(COUNT(track_name) AS FLOAT) / (SELECT COUNT(track_name)
                                                   FROM invoice_genre), 2) pct_tracks_sold
  FROM invoice_genre
 GROUP BY genre
 ORDER BY num_tracks_sold DESC
 LIMIT 10;

 * sqlite:///chinook.db
Done.


genre,num_tracks_sold,pct_tracks_sold
Rock,2635,0.55
Metal,619,0.13
Alternative & Punk,492,0.1
Latin,167,0.04
R&B/Soul,159,0.03
Blues,124,0.03
Jazz,121,0.03
Alternative,117,0.02
Easy Listening,74,0.02
Pop,63,0.01


When looking at the whole Chinook store, rock music is still at the top of our leaderboard. However, metal music seems to be slightly more popular than alternative & punk music at global level. Interestingly, pop music (which is short for popular music) ranks at the bottom.

## 3
## Which employee performs the best?

**Scenario**

> Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

**Question**

What's the performance of each sales support agent?

In [5]:
%%sql

WITH invoice_by_employee AS (SELECT i.invoice_id,
                                    i.total,
                                    c.country AS customer_country,
                                    c.customer_id,
                                    c.support_rep_id,
                                    e.first_name || " " || e.last_name AS employee_name,
                                    e.hire_date,
                                    e.country AS employee_country
                               FROM invoice i
                                    INNER JOIN customer c ON i.customer_id = c.customer_id
                                    INNER JOIN employee e ON c.support_rep_id = e.employee_id)

SELECT employee_name, 
       ROUND(SUM(total), 2) AS total_sales,
       COUNT(DISTINCT(customer_id)) AS num_customers,
       hire_date
  FROM invoice_by_employee
 GROUP BY employee_name
 ORDER BY total_sales DESC;

 * sqlite:///chinook.db
Done.


employee_name,total_sales,num_customers,hire_date
Jane Peacock,1731.51,21,2017-04-01 00:00:00
Margaret Park,1584.0,20,2017-05-03 00:00:00
Steve Johnson,1393.92,18,2017-10-17 00:00:00


**Conclusion**

Jane Peacock has the best performance by total sales. However, it's worth noting that Jane has been hired the longest among 3 sales support agents. For better analysis, we could find average monthly sales generated by each agent and compare those numbers. 

Apart from hiring date, we also check number of customers assigned to each agent. It seems that total sales positively correlates with number of customers. The agent with the highest total sales has the highest number of customers assigned to, whereas the agent with the lowest total sales has the lowest number of customers.

## 4
## Which country generates the most sales to Chinook?

In [6]:
%%sql

WITH customer_country AS (SELECT customer_id,
                                 country,
                                 COUNT(DISTINCT(customer_id)) AS num_customers,
                                 CASE
                                    WHEN COUNT(DISTINCT(customer_id)) = 1 THEN 'Other'
                                    ELSE country
                                    END AS new_country
                            FROM customer
                        GROUP BY country),

     invoice_country AS (SELECT i.*,
                                cc.new_country
                           FROM invoice i
                                LEFT JOIN customer c ON i.customer_id = c.customer_id
                                LEFT JOIN customer_country cc ON c.country = cc.country)

SELECT new_country AS country,
       COUNT(DISTINCT(customer_id)) AS num_customers,
       ROUND(SUM(total),2) AS total_sales,
       ROUND(SUM(total) / COUNT(DISTINCT(customer_id)), 2) AS avg_sales_per_customer,
       COUNT(DISTINCT(invoice_id)) AS num_orders,
       ROUND(SUM(total) / COUNT(DISTINCT(invoice_id)), 2) AS avg_order_value
  FROM (SELECT *, 
        CASE
            WHEN new_country = 'Other' THEN 1
            ELSE 0
            END AS sort
        FROM invoice_country)
 GROUP BY new_country
 ORDER BY sort ASC, total_sales DESC;

 * sqlite:///chinook.db
Done.


country,num_customers,total_sales,avg_sales_per_customer,num_orders,avg_order_value
USA,13,1040.49,80.04,131,7.94
Canada,8,535.59,66.95,76,7.05
Brazil,5,427.68,85.54,61,7.01
France,5,389.07,77.81,50,7.78
Germany,4,334.62,83.66,41,8.16
Czech Republic,2,273.24,136.62,30,9.11
United Kingdom,3,245.52,81.84,28,8.77
Portugal,2,185.13,92.57,29,6.38
India,2,183.15,91.58,21,8.72
Other,15,1094.94,73.0,147,7.45


USA has the highest total sales but average value of sales per customer or per order is not as high as Czech Republic or India. Chinook should focus on retaining the customers while trying to increase sales per order/customer, to boost overall sales.

Meanwhile, given current number of customers in Czech Republic and India, there are definitely opportunities for Chinook to expand the business and reach more customers while try to maintain current sales per order/customer.

## 5
## Which one is better: Albums vs Individual Tracks?

**Scenario**

>The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

>1. purchase a whole album
2. purchase a collection of one or more individual tracks.

>The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

>Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

**Question**

Should the management change the purchasing strategy?

In [7]:
%%sql

WITH 
     /* Find number of tracks in each album */
     track_per_album AS (SELECT a.album_id,
                                a.title,
                                COUNT(t.track_id) AS num_tracks
                           FROM track t
                                INNER JOIN album a ON t.album_id = a.album_id
                          GROUP BY a.album_id
                          ORDER BY a.album_id),
     
     /* Find number of tracks of each album in each invoice */
     track_album_invoice AS (SELECT il.invoice_id,
                                    a.album_id,
                                    COUNT(il.track_id) AS num_tracks
                               FROM invoice_line il
                                    INNER JOIN track t ON il.track_id = t.track_id
                                    INNER JOIN album a ON t.album_id = a.album_id
                              GROUP BY il.invoice_id, a.album_id
                              ORDER BY il.invoice_id, a.album_id),

     /* Determine if an invoice is an album purchase */
     album_purchases AS (SELECT tai.invoice_id,
                                tai.album_id,
                                tai.num_tracks,
                                tpa.num_tracks AS num_tracks_album,
                                CASE
                                    WHEN 
                                        ((tai.num_tracks = tpa.num_tracks)
                                        AND
                                        (COUNT(DISTINCT(tai.album_id)))=1)
                                    THEN "album_purchase"
                                    ELSE "single_track_purchase"
                                    END AS "purchase_type"
                           FROM track_album_invoice tai
                                LEFT JOIN track_per_album tpa ON tai.album_id = tpa.album_id
                          GROUP BY invoice_id)

SELECT purchase_type,
       COUNT(DISTINCT(invoice_id)) AS num_invoices,
       ROUND(CAST(COUNT(DISTINCT(invoice_id)) AS FLOAT) / (SELECT COUNT(*) 
                                                            FROM invoice), 2) AS pct_invoices
  FROM album_purchases
 GROUP BY purchase_type;

 * sqlite:///chinook.db
Done.


purchase_type,num_invoices,pct_invoices
album_purchase,114,0.19
single_track_purchase,500,0.81


Over 80% of all invoices are single-track purchases. I would suggest that Chinook management changes their purchasing strategy going forward to only purchasing the popular tracks from each album.

## 6
## How many tracks have been purchased vs not purchased?

In [8]:
%%sql

SELECT (CASE
            WHEN EXISTS (SELECT * FROM invoice_line il
                          WHERE il.track_id = t.track_id)
                 THEN "purchased"
            ELSE "not purchased"
            END) AS purchase_status,
        COUNT(track_id) AS num_tracks,
        ROUND(CAST(COUNT(track_id) AS FLOAT) / (SELECT COUNT(*) 
                                                  FROM track), 2) AS pct_track_purchased
  FROM track t
 GROUP BY purchase_status;

 * sqlite:///chinook.db
Done.


purchase_status,num_tracks,pct_track_purchased
not purchased,1697,0.48
purchased,1806,0.52


Almost half of the tracks has never been purchased. This strengthens our previous suggestion that Chinook management should change the purchasing strategy in order to avoid carrying non-revenue-generating tracks on their server.

## 7
## Which artist is used in the most playlists?

In [9]:
%%sql

WITH playlist_info AS (SELECT p.playlist_id,
                              a.name AS artist
                         FROM playlist p
                              INNER JOIN playlist_track pt ON p.playlist_id = pt.playlist_id
                              INNER JOIN track t ON pt.track_id = t.track_id
                              INNER JOIN album ON t.album_id = album.album_id
                              INNER JOIN artist a ON album.artist_id = a.artist_id)

SELECT artist,
       COUNT(DISTINCT(playlist_id)) AS num_playlists
  FROM playlist_info
 GROUP BY artist
 ORDER BY num_playlists DESC
 LIMIT 10;

 * sqlite:///chinook.db
Done.


artist,num_playlists
Eugene Ormandy,7
The King's Singers,6
English Concert & Trevor Pinnock,6
Berliner Philharmoniker & Herbert Von Karajan,6
Academy of St. Martin in the Fields & Sir Neville Marriner,6
Yo-Yo Ma,5
Wilhelm Kempff,5
Ton Koopman,5
"Sir Georg Solti, Sumi Jo & Wiener Philharmoniker",5
Sir Georg Solti & Wiener Philharmoniker,5


The artist that appears the most in the playlist is *Eugene Ormandy*. He appears in 7 playlists. 

## 8
## Which track is being purchased the most?

We will find out which track is being purchased the most. We will count how many times the track appears on invoices and check the quantity being invoiced.

In [10]:
%%sql

WITH invoice_track AS (SELECT il.invoice_id,
                              t.name AS track_name,
                              g.name AS genre,
                              album.title AS album_name,
                              a.name AS artist_name,
                              il.quantity
                         FROM invoice_line il
                              INNER JOIN track t ON il.track_id = t.track_id
                              INNER JOIN genre g ON t.genre_id = g.genre_id
                              INNER JOIN album ON t.album_id = album.album_id
                              INNER JOIN artist a ON album.artist_id = a.artist_id)

SELECT track_name,
       genre,
       album_name,
       artist_name,
       COUNT(DISTINCT invoice_id) AS num_invoices,
       SUM(quantity) AS quantity
  FROM invoice_track
 GROUP BY track_name
 ORDER BY num_invoices DESC
 LIMIT 10;

 * sqlite:///chinook.db
Done.


track_name,genre,album_name,artist_name,num_invoices,quantity
War Pigs,Alternative,Cake: B-Sides and Rarities,Cake,33,33
Highway Chile,Rock,Are You Experienced?,Jimi Hendrix,14,14
Changes,Metal,Black Sabbath Vol. 4 (Remaster),Black Sabbath,14,14
Are You Experienced?,Rock,Are You Experienced?,Jimi Hendrix,14,14
Third Stone From The Sun,Rock,Are You Experienced?,Jimi Hendrix,13,13
Put The Finger On You,Rock,For Those About To Rock We Salute You,AC/DC,13,13
Hey Joe,Rock,Are You Experienced?,Jimi Hendrix,13,13
We Are The Champions,Rock,Greatest Hits I,Queen,12,12
Love Or Confusion,Rock,Are You Experienced?,Jimi Hendrix,12,12
Drain You,Rock,From The Muddy Banks Of The Wishkah [live],Nirvana,12,12


Although *Eugene Ormandy* is added to the most playlists, the track that is most-purchased is *War Pigs* by Cake. The song is purchased 33 times. Tracks from *Jimi Hendrix* seem to receive lots of love from customers as 5 out of top 10 most-purchased tracks are his songs.

## 9
## Which artist sells the most songs?

In [11]:
%%sql

WITH invoice_track AS (SELECT il.invoice_id,
                              t.name AS track_name,
                              g.name AS genre,
                              album.title AS album_name,
                              a.name AS artist_name,
                              il.quantity
                         FROM invoice_line il
                              INNER JOIN track t ON il.track_id = t.track_id
                              INNER JOIN genre g ON t.genre_id = g.genre_id
                              INNER JOIN album ON t.album_id = album.album_id
                              INNER JOIN artist a ON album.artist_id = a.artist_id)

SELECT artist_name,
       genre,
       SUM(quantity) AS quantity,
       COUNT(DISTINCT album_name) AS num_albums,
       COUNT(DISTINCT track_name) AS num_tracks
  FROM invoice_track
GROUP BY artist_name
ORDER BY quantity DESC
LIMIT 10;

 * sqlite:///chinook.db
Done.


artist_name,genre,quantity,num_albums,num_tracks
Queen,Rock,192,3,43
Jimi Hendrix,Rock,187,1,17
Red Hot Chili Peppers,Rock,130,3,48
Nirvana,Rock,130,2,24
Pearl Jam,Rock,129,5,58
Guns N' Roses,Rock,124,3,42
AC/DC,Rock,124,2,18
Foo Fighters,Rock,121,4,43
The Rolling Stones,Rock,117,3,39
Metallica,Metal,106,10,68


The songs from *Queens* are sold the most, at 192 times from 43 tracks. *Jimi Hendrix* ranks the second at 187 times from 17 tracks. Out of our top 10, 9 artists offer rock music. This is also aligned with our first finding that rock music is the most popular genre in Chinook store.